In [1]:
%matplotlib inline

import cv2
import itertools as it
import math
import matplotlib.pyplot as plt
import numpy as np
import os
import pickle
import random
from utilities import *

In [2]:
images = list(pickle_iter(r"D:\Valorant\Killed\Color\00001.pickle"))
len(images)

8000

In [3]:
for i, image in enumerate(images):
    if show_and_wait(image, wait_time=1) == 'q':
        print(i)
        break
cv2.destroyAllWindows()

In [4]:
upper_positives, lower_positives = images[6096:6296], images[7748:7998]
positives = upper_positives + lower_positives
def fn(image):
    return cv2.resize(image, (30, 30))
g = grouper(positives, 25)
g = (np.hstack(list(map(fn, g))) for g in g)
show_and_wait(np.vstack(list(g)))
cv2.destroyAllWindows()
list(prime_factors(len(positives)))

[2, 3, 3, 5, 5]

In [5]:
for i, image in enumerate(positives):
    if show_and_wait(image, wait_time=25) == 'q':
        break
cv2.destroyAllWindows()

In [6]:
left, right = 3, 59
width, height = right - left, 11
upper_extent = (59, left, 59 + height, right)
lower_extent = (95, left, 95 + height, right)
g = ((upper_positives, upper_extent), (lower_positives, lower_extent))
g = (zip(a, it.repeat(b)) for a, b in g)
g = it.chain.from_iterable(g)
g = (i[top:bottom, left:right] for i, (top, left, bottom, right) in g)
g = grouper(g, 10)
g = (np.hstack(list(g)) for g in g)
show_and_wait(np.vstack(list(g)))
cv2.destroyAllWindows()
(upper_extent[2] - upper_extent[0], upper_extent[3] - upper_extent[1],
 lower_extent[2] - lower_extent[0], lower_extent[3] - lower_extent[1])

(11, 56, 11, 56)

In [7]:
g = ((upper_positives, upper_extent), (lower_positives, lower_extent))
g = (zip(a, it.repeat(b)) for a, b in g)
g = it.chain.from_iterable(g)
g = (i[top:bottom, left:right] for i, (top, left, bottom, right) in g)
color = np.stack(list(g)).mean(axis=(0, 1, 2))
show_and_wait(np.broadcast_to(color, (222, 222, 3)))
cv2.destroyAllWindows()
color.shape, color

((3,), array([0.5691015, 0.6987058, 0.7345293], dtype=float32))

In [8]:
template = np.broadcast_to(color, (height, width, 3))
match = cv2.matchTemplate(upper_positives[0][:, left:right], template, cv2.TM_SQDIFF)
show_and_wait(np.broadcast_to(match / match.max(), (match.shape[0], 222)))
cv2.destroyAllWindows()
match.reshape(-1), match.min(), match.max()

(array([128.6426  , 127.5322  , 126.27965 , 124.479324, 122.508446,
        120.03372 , 117.15281 , 113.81479 , 110.16839 , 106.23877 ,
        102.02189 ,  97.77    ,  93.92924 ,  90.34311 ,  87.257164,
         84.179245,  81.32891 ,  78.797295,  76.677055,  74.83733 ,
         73.34904 ,  72.17568 ,  71.07899 ,  69.86489 ,  68.81485 ,
         67.67303 ,  66.589264,  65.36002 ,  64.25471 ,  63.069504,
         61.682972,  60.015312,  58.082607,  56.29231 ,  54.69686 ,
         53.05775 ,  51.70887 ,  50.49536 ,  49.435486,  48.319233,
         47.3587  ,  46.602898,  46.04036 ,  45.617027,  45.502365,
         45.366463,  44.49129 ,  41.370388,  38.154526,  35.01607 ,
         31.871313,  28.760847,  25.70094 ,  22.658968,  19.793331,
         16.611198,  13.406542,  11.025362,  10.860258,  10.809429,
         11.071693,  12.512628,  31.637377,  51.375805,  71.55317 ,
         91.3508  , 109.59195 , 127.389824, 146.61267 , 164.92969 ,
        181.7354  , 207.4926  , 228.01678 , 230.

In [9]:
negatives = random.sample(range(6000), 340) + random.sample(range(6310, 7745), 110)
negatives = [images[i] for i in sorted(negatives)]
g = (np.hstack([cv2.resize(image, (55, 55)) for image in g]) for g in grouper(negatives, 25))
show_and_wait(np.vstack(list(g)))
cv2.destroyAllWindows()

In [10]:
len(positives), positives[0].shape, len(negatives), negatives[0].shape

(450, (108, 168, 3), 450, (108, 168, 3))

In [11]:
def fn(image):
    match = cv2.matchTemplate(image[:, left:right], template, cv2.TM_SQDIFF)
    return match
image = np.hstack(list(map(fn, negatives + positives)))
show_and_wait(image / image.max())
cv2.destroyAllWindows()

In [12]:
def fn(image):
    n, d = 25, math.log(2)
    match = cv2.matchTemplate(cv2.cvtColor(image[:, left:right], cv2.COLOR_BGR2LUV), cv2.cvtColor(template, cv2.COLOR_BGR2LUV), cv2.TM_SQDIFF)
    if image.max() == image.min() == 1:
        print('-')
    else:
        print(match.min(), match.max(), round(math.log(match.min(), 2)), round(math.log(match.max(), 2)), len([v for v in match.reshape(-1) if v < 2*match.min()]),
              match.min() < 200000)#500000 and len([v for v in match.reshape(-1) if v < 2*match.min()]) < 70)
    match = np.clip(match, 0, 2**18)
    match = np.broadcast_to(match / match.max(), (match.shape[0], n))
    return np.hstack([cv2.cvtColor(cv2.resize(match, (n, n)), cv2.COLOR_GRAY2BGR), cv2.resize(image, (n, n))])
g = grouper(it.chain(negatives, 25 * [np.ones_like(negatives[0])], positives), 25)
g = (np.hstack(list(map(fn, g))) for g in g)
list(g)#show_and_wait(np.vstack(list(g)))
cv2.destroyAllWindows()

664952.44 1954082.8 19 21 38 False
1143504.5 1954082.8 20 21 98 False
676506.25 1903990.6 19 21 39 False
674853.7 1903990.6 19 21 39 False
690872.8 1916141.1 19 21 40 False
690811.2 1905965.4 19 21 40 False
690811.2 1905965.4 19 21 40 False
672233.6 1911845.5 19 21 39 False
700966.75 1927937.1 19 21 44 False
717655.25 1916532.4 19 21 47 False
1202959.8 1912218.5 20 21 98 False
1198605.6 1912730.6 20 21 98 False
1201127.6 1940632.9 20 21 98 False
1187724.1 1945740.6 20 21 98 False
1188446.6 1945068.6 20 21 98 False
1151839.6 1903263.6 20 21 98 False
1151887.5 1901828.9 20 21 98 False
1152462.1 1903263.6 20 21 98 False
1157052.1 1902618.1 20 21 98 False
1173524.8 1918533.1 20 21 98 False
1173924.4 1918509.8 20 21 98 False
1147879.2 1915736.5 20 21 98 False
1162822.1 1925088.6 20 21 98 False
1162118.1 1925120.2 20 21 98 False
1131461.4 1909209.2 20 21 98 False
1158085.9 1908942.6 20 21 98 False
1163317.0 1915416.2 20 21 98 False
1164378.0 1915586.0 20 21 98 False
1164795.1 1915586.0 20 21

279659.38 307549.38 18 18 98 False
413211.34 451246.16 19 19 98 False
1413951.8 1603501.4 20 21 98 False
1158108.5 1756694.5 20 21 98 False
1392064.5 1757732.9 20 21 98 False
722131.56 806653.1 19 20 98 False
509232.78 608476.75 19 19 98 False
1022449.1 1121756.1 20 20 98 False
1077378.6 1502840.0 20 21 98 False
1083356.1 1826104.8 20 21 98 False
1152107.2 2128050.0 20 21 98 False
525453.5 587521.06 19 19 98 False
359427.1 547198.4 18 19 98 False
1646137.1 2782588.2 21 21 98 False
614751.25 1002451.6 19 20 98 False
314966.88 2165753.5 18 21 16 False
1252619.0 1704875.2 20 21 98 False
594939.7 705221.2 19 19 98 False
121333.69 166465.02 17 17 98 True
161382.73 200866.9 17 18 98 True
157253.81 529970.3 17 19 74 True
139030.6 575402.3 17 19 58 True
265032.38 1176415.9 18 20 84 False
143483.95 775825.3 17 20 36 True
50393.55 139971.52 16 17 95 True
37190.363 99703.02 15 17 27 True
91689.016 126058.91 16 17 98 True
76768.03 162113.45 16 17 93 True
120097.234 134460.22 17 17 98 True
16272.67

134007.11 1761029.1 17 21 63 True
136704.58 1761029.1 17 21 63 True
125917.35 1761029.1 17 21 61 True
119781.516 1761029.1 17 21 37 True
120469.84 1761029.1 17 21 39 True
124330.58 1761029.1 17 21 53 True
121463.15 1761029.1 17 21 39 True
120205.12 1761029.1 17 21 37 True
126084.055 1761029.1 17 21 57 True
124650.445 1761029.1 17 21 50 True
123821.445 1761029.1 17 21 45 True
126094.75 1765041.8 17 21 55 True
127895.88 1765041.8 17 21 63 True
128196.21 1765041.8 17 21 63 True
124937.76 1765041.8 17 21 54 True
124732.34 1765041.8 17 21 50 True
124411.07 1765041.8 17 21 47 True
123941.16 1765041.8 17 21 47 True
127815.07 1765041.8 17 21 62 True
128762.195 1765041.8 17 21 63 True
124927.14 1765041.8 17 21 53 True
125451.6 1765041.8 17 21 52 True
121959.14 1763996.0 17 21 37 True
120368.07 1763996.0 17 21 37 True
121426.82 1763996.0 17 21 37 True
122625.08 1763996.0 17 21 46 True
122405.336 1763996.0 17 21 51 True
123347.13 1763996.0 17 21 46 True
123859.96 1763996.0 17 21 54 True
123070.80